In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import json

# .env 파일 로드
load_dotenv()

In [14]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
%%writefile parse.yaml

modules:
  - module_type: langchain_parse
    parse_method: pypdf
    file_type: all_files
    language: ko

In [ ]:
# make project folder
import os
os.makedirs('./parse_project_dir')

In [ ]:
from autorag.parser import Parser

parser = Parser(data_path_glob="./pdf_data/(1) 2024 달라지는 세금제도.pdf", project_dir="./parse_project_dir")
parser.start_parsing("./parse.yaml")

In [ ]:
# Let's investigate how Raw looks like.
# First open summary.csv file
import pandas as pd
parse_summary_df = pd.read_csv("./parse_project_dir/summary.csv") # The folder path must be changed up to v0.3.8
parse_summary_df

In [ ]:
# 0.parquet is langchain_parse pdfminer. So, let's open 0.parquet first.
pdfminer_raw_df = pd.read_parquet("./parse_project_dir/pdf.parquet") # The folder path must be changed up to v0.3.8
pdfminer_raw_df.head()

In [ ]:
%%writefile chunk.yaml

modules:
  - module_type: llama_index_chunk
    chunk_method: [ Token, Sentence ]
    chunk_size: [ 1024, 512 ]
    chunk_overlap: 50
    add_file_name: ko

In [ ]:
from autorag.chunker import Chunker

chunker = Chunker.from_parquet(parsed_data_path="./parse_project_dir/pdf.parquet", project_dir="./chunk_project_dir")
chunker.start_chunking("./chunk.yaml")

In [ ]:
# Let's investigate how chunked corpus result looks like.
# First open summary.csv file
import pandas as pd
chunk_summary_df = pd.read_csv("./chunk_project_dir/summary.csv") # The folder path must be changed up to v0.3.8
chunk_summary_df

In [23]:
from autorag.data.qa.schema import Raw, Corpus

raw_df = pd.read_parquet("./chunk_project_dir/0.parquet") # The folder path must be changed up to v0.3.8
raw_instance = Raw(raw_df)

corpus_df = pd.read_parquet("./chunk_project_dir/0.parquet") # The folder path must be changed up to v0.3.8orpus_instance = Corpus(corpus_df, raw_instance)
corpus_instance = Corpus(corpus_df, raw_instance)


In [ ]:
import pandas as pd
from llama_index.llms.openai import OpenAI

from autorag.data.qa.filter.dontknow import dontknow_filter_rule_based
from autorag.data.qa.generation_gt.llama_index_gen_gt import (
    make_basic_gen_gt,
    make_concise_gen_gt,
)
from autorag.data.qa.query.llama_gen_query import factoid_query_gen
from autorag.data.qa.sample import random_single_hop

llm = OpenAI(model="gpt-4o-mini")

initial_qa = (
    corpus_instance.sample(random_single_hop, n=20) # Only sample three passages. This will make three QA dataset only. Needs to adjust before real use.
    .map(
        lambda df: df.reset_index(drop=True), # Drop index
    )
    .make_retrieval_gt_contents() # This is essential for making questions with gt (ground truth) passages.
    .batch_apply(
        factoid_query_gen,  # query generation
        llm=llm,lang='ko'
    )
    .batch_apply(
        make_basic_gen_gt,  # answer generation (basic)
        llm=llm,lang='ko'
    )
    .batch_apply(
        make_concise_gen_gt,  # answer generation (concise)
        llm=llm,lang='ko'
    )
    .filter(
        dontknow_filter_rule_based,  # unanswerable filter
        lang='ko'
    )
)

initial_qa.to_parquet('./initial_qa_0.parquet', './initial_corpus_0.parquet')

In [29]:
from autorag.data.qa.schema import Raw, Corpus

raw_df = pd.read_parquet("./chunk_project_dir/1.parquet") # The folder path must be changed up to v0.3.8
raw_instance = Raw(raw_df)

corpus_df = pd.read_parquet("./chunk_project_dir/1.parquet") # The folder path must be changed up to v0.3.8orpus_instance = Corpus(corpus_df, raw_instance)
corpus_instance = Corpus(corpus_df, raw_instance)


In [ ]:
import pandas as pd
from llama_index.llms.openai import OpenAI

from autorag.data.qa.filter.dontknow import dontknow_filter_rule_based
from autorag.data.qa.generation_gt.llama_index_gen_gt import (
    make_basic_gen_gt,
    make_concise_gen_gt,
)
from autorag.data.qa.query.llama_gen_query import factoid_query_gen
from autorag.data.qa.sample import random_single_hop

llm = OpenAI(model="gpt-4o-mini")

initial_qa2 = (
    corpus_instance.sample(random_single_hop, n=60) # Only sample three passages. This will make three QA dataset only. Needs to adjust before real use.
    .map(
        lambda df: df.reset_index(drop=True), # Drop index
    )
    .make_retrieval_gt_contents() # This is essential for making questions with gt (ground truth) passages.
    .batch_apply(
        factoid_query_gen,  # query generation
        llm=llm,lang='ko'
    )
    .batch_apply(
        make_basic_gen_gt,  # answer generation (basic)
        llm=llm,lang='ko'
    )
    .batch_apply(
        make_concise_gen_gt,  # answer generation (concise)
        llm=llm,lang='ko'
    )
    .filter(
        dontknow_filter_rule_based,  # unanswerable filter
        lang='ko'
    )
)

initial_qa2.to_parquet('./initial_qa_1.parquet', './initial_corpus_1.parquet')

In [ ]:
# Combine all QA datasets
qa_dfs = []
corpus_dfs = []

for i in range(2):
    qa_path = f'./initial_qa_{i}.parquet'
    corpus_path = f'./initial_corpus_{i}.parquet'
    
    if os.path.exists(qa_path) and os.path.exists(corpus_path):
        qa_df = pd.read_parquet(qa_path)
        corpus_df = pd.read_parquet(corpus_path)
        qa_dfs.append(qa_df)
        corpus_dfs.append(corpus_df)

# Concatenate all dataframes
combined_qa = pd.concat(qa_dfs, ignore_index=True)
combined_corpus = pd.concat(corpus_dfs, ignore_index=True)

# Save combined datasets
combined_qa.to_parquet('./combined_qa.parquet')
combined_corpus.to_parquet('./combined_corpus.parquet')


In [35]:
combined_qa.to_csv('rag_val_new.csv',index=False, encoding='utf-8-sig')

In [38]:
combined_qa

,qid,query,retrieval_gt,generation_gt
0,9befbcf1-b163-4400-8daf-e0891944fb24,2024년 1월 1일부터 적용되는 세금제도의 주요 내용은 무엇입니까?,[[6cf37366-43cf-44a0-8eac-b112f786daa4]],"[2024년 1월 1일부터 적용되는 세금제도의 주요 내용은 비과세 및 면제, 제한세..."
1,12788d22-e80a-45a2-8d40-23137628dc2f,상속재산 평가방법 차이에 대한 납부지연가산세는 어떻게 적용됩니까?,[[da563dcb-9543-42ac-a609-3bfecf2282dd]],[상속재산 평가방법 차이에 대한 납부지연가산세는 부과되지 않게 되었습니다. 이는 과...
2,38cd981d-6892-4443-b0eb-4f19068cb204,2024년까지 연장되는 세액공제의 적용기한은 언제까지입니까?,[[e6e778f0-a9c0-4c45-8595-6c12c20124bc]],"[2024년까지 연장되는 세액공제의 적용기한은 2024년 12월 31일까지입니다.,..."
3,fd59f7c6-b58e-4c73-a8db-8102b19594d8,2024년부터 적용되는 기타소득에 대한 간이지급명세서 제출 규정은 무엇입니까?,[[3dd44d00-20d6-4d86-9b17-cecf944048d8]],"[2024년부터 적용되는 기타소득에 대한 간이지급명세서 제출 규정은, 인적용역 관련..."
4,62070ee1-7e93-4b38-8251-540269b79517,2023년 7월에 발표된 세법 개정안은 어떤 내용을 중심으로 하고 있습니까?,[[f85a35fc-2d42-4b46-bf34-08e652546946]],[2023년 7월에 발표된 세법 개정안은 '2023 핵심 개정세법'과 관련된 내용을...
...,...,...,...,...
75,967b1943-6e84-4773-8a3f-e16c082e6827,2024년 1월 1일부터 적용되는 양도소득세의 변화는 무엇입니까?,[[74ee3323-ccac-45c6-9a0a-5c827af87c54]],[2024년 1월 1일부터 적용되는 양도소득세의 변화는 현물출자에 대한 규정입니다....
76,a80161c4-dd0a-4c24-b907-304b961b7ce8,지역특구 세액감면의 적용기한은 언제까지 연장되었습니까?,[[66e3939a-11f0-442c-a238-f257b22e9364]],"[지역특구 세액감면의 적용기한은 2025년 12월 31일까지 연장되었습니다., 20..."
77,9435f5a2-579e-4a28-8a4e-637438e1ccf2,현금영수증사업자에 대한 부가가치세 과세특례 적용기한은 언제까지였습니까?,[[75108738-64b7-4a17-a579-03a7549c169a]],[현금영수증사업자에 대한 부가가치세 과세특례 적용기한은 2025년 12월 31일까지...
78,d9a513eb-f77e-43ad-9a03-b208ff9da19b,2024년 1월 1일부터 적용되는 세금제도의 주요 내용은 무엇입니까?,[[5964b15a-2018-4dfc-b156-236fee329d12]],[2024년 1월 1일부터 적용되는 세금제도의 주요 내용은 다음과 같습니다:\n\n...


In [23]:
import pandas as pd
df = pd.read_csv('rag_val_new_post.csv')

In [ ]:
df['generation_gt'] = df['generation_gt'].apply(lambda x : x[2:])
df.to_csv('rag_val_new_post.csv',index=False, encoding='utf-8-sig')
df